# MIE 1624 Rating Recommendation System
- Group 8


We will divide it into three main parts:


* 1)Data Cleaning

* 2)Model Implementation

* 3)Test Prediction


import required packages:

In [ ]:
# If running on GoogleColab
!pip install whoosh

In [ ]:
# If running on Google Colab
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train (1).csv
Saving rating_pairs.csv to rating_pairs.csv


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import re, nltk
from nltk.stem import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from whoosh import index, writing, scoring
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os, os.path
import shutil
from collections import Counter
import string
from wordcloud import WordCloud
import seaborn as sns
#################################################################
from html.parser import HTMLParser
from pandas import DataFrame
#################################################################
nltk.download('averaged_perceptron_tagger')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
#################################################################
from nltk.stem import PorterStemmer



##################################################################
import copy
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

##################################################################
import html 
import collections 
from collections import Counter

##################################################################
import warnings
warnings.filterwarnings("ignore")

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 1. Data Processing

We have two files: train.csv and test.csv. We will use train.csv to build and train a model used for predicting the rating of the amazon product.

We can see that there are 11 columns in the train.csv while 10 columns in the test.csv which doesn't have the column called overall. 

To prepare the features used for prediction. we will select the column summary and review text as our main raw data. Afterwards we will use NLP to process extract the important words using different methods

The procedure will follows as below:



### 1.1 Necessary Functions used for data cleaning

In [ ]:
def remove_nonAlphaNum(tweet):
    return re.sub(r'[^\s\w]+', '', str(tweet))

def removeWhiteSpace(tweet):
    tweet = str(tweet)
    return tweet.replace('\r', '').replace('\n', '')

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    sentence = str(sentence)
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize((sentence)))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

### 1.2 Train Data Processing

In [ ]:
# Colab use the first df
# anaconda use the second
#import io
#df = pd.read_csv(io.BytesIO(uploaded['train.csv']))
df = pd.read_csv('train (1).csv', error_bad_lines = False)#,engine='python')
df.head()

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,Jazz,$8.40,p23649501,3856620,NaN
1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,Alternative Rock,$10.98,p58458313,56086781,NaN
2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,Pop,$5.99,p97027626,55852154,NaN
3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,Jazz,$14.64,p43167086,43228100,NaN
4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,Alternative Rock,$9.92,p94494236,54425467,NaN


In [ ]:
df_overall = df[["overall"]]
df["text"] = df["reviewText"] + df["summary"]      #combine summary and reviewText
df["text"] = df["text"].str.lower()                #lowercase
df

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image,text
0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,Jazz,$8.40,p23649501,3856620,NaN,a contemporary jazz and soul performer who's m...
1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,Alternative Rock,$10.98,p58458313,56086781,NaN,very good idea to put both the 'pop' and 'orch...
2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,Pop,$5.99,p97027626,55852154,NaN,this is a great collection of carole king's so...
3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,Jazz,$14.64,p43167086,43228100,NaN,the is album is a brilliant piece of jazz fusi...
4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,Alternative Rock,$9.92,p94494236,54425467,NaN,waited a long time for this dvd to be released...
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,5.0,"09 12, 2013",u29136602,There is just something about Wesley's voice t...,Just right.,1378944000,Alternative Rock,$9.49,p26326919,90575108,NaN,there is just something about wesley's voice t...
149996,5.0,"05 15, 2016",u51715193,How many recordings of Verdis Requiem do you o...,Being there in vibrant immediacy of performan...,1463270400,Classical,$14.83,p69247882,74425587,NaN,how many recordings of verdis requiem do you o...
149997,5.0,"06 13, 2004",u28696060,....their best overall album. After the revis...,Creative rebirth and possibly....,1087084800,Alternative Rock,$18.86,p07028930,20399056,NaN,....their best overall album. after the revis...
149998,5.0,"06 27, 2012",u04925906,"Believe it or not, once upon a time, artists l...",Once Upon A Time,1340755200,Pop,$19.99,p38606785,5931588,NaN,"believe it or not, once upon a time, artists l..."


In [ ]:
df["text"] = df["text"].apply(remove_nonAlphaNum)         #remove commas and everythin
df["text"] = df["text"].apply(removeWhiteSpace)           # remove line breaks and tabs

In [ ]:
# 14 min 11 seconds under GPU and large RA; Google Colab
lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')
# Lemmatizing
df["text"] = df["text"].apply(lambda x: lemmatize_sentence(x))
print(df.head())

   overall  ...                                               text
0      5.0  ...  a contemporary jazz and soul performer who mak...
1      5.0  ...  very good idea to put both the pop and orchest...
2      5.0  ...  this be a great collection of carole king song...
3      5.0  ...  the be album be a brilliant piece of jazz fusi...
4      5.0  ...  wait a long time for this dvd to be release ha...

[5 rows x 12 columns]


In [ ]:
#remove stopwords
stop_words = stopwords.words('english')
df["text"]= df["text"].apply(lambda x : ' '.join([word for word in x.split(' ') if word not in stop_words]))

In [ ]:
#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['text']= df['text'].apply(lambda x : ' '.join(st.stem(i) for i in x.split()))

In [ ]:
## remove single word character
df["text"]= df["text"].apply(lambda x : ' '.join(i for i in x.split() if not (i.isalpha() and len(i)==1)))
## remove numbers
df["text"]= df["text"].apply(lambda x : re.sub(r'\w*\d+\w*', '', x))

In [ ]:
text = df[["text"]]
clean_train = df[["text","overall"]]
clean_train.to_csv('clean_train.csv', sep = ',',index = False)
print(clean_train.head())
print(np.shape(clean_train))

                                                text  overall
0  contemporari jazz soul perform make twodecad c...      5.0
1  good idea put pop orchestr music matrix reload...      5.0
2  great collect carol king songsa musthav carol ...      5.0
3  album brilliant piec jazz fusion start finish ...      5.0
4  wait long time dvd releas watch  yearold vh di...      5.0
(150000, 2)


### 1.3 Test Data Processing

In [ ]:
#read data 
df_test = pd.read_csv("test.csv")       
#combine summary and reviewText
df_test["text"] = df_test["reviewText"] + df_test["summary"] 
#lowercase
df_test["text"] = df_test["text"].str.lower() 
#remove commas and everythin
df_test["text"] = df_test["text"].apply(remove_nonAlphaNum)
# remove line breaks and tabs
df_test["text"] = df_test["text"].apply(removeWhiteSpace)           
#lemmatize
df_test["text"] = df_test["text"].apply(lambda x: lemmatize_sentence(x))
#remove stopwords
stop_words = stopwords.words('english')
df_test["text"]= df_test["text"].apply(lambda x : ' '.join([word for word in x.split(' ') if word not in stop_words]))
#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df_test['text']= df_test['text'].apply(lambda x : ' '.join(st.stem(i) for i in x.split()))
## remove single word character
df_test["text"]= df_test["text"].apply(lambda x : ' '.join(i for i in x.split() if not (i.isalpha() and len(i)==1)))
## remove numbers
df_test["text"]= df_test["text"].apply(lambda x : re.sub(r'\w*\d+\w*', '', x))


# out put clean data
clean_test = df_test[["text"]]
clean_test.to_csv('clean_test.csv', sep = ',',index = False)
print(clean_test.head())
print(np.shape(clean_test))

                                                text
0                                         okfiv star
1    set alon worth purchas cost everyth els grav...
2  mari blige someon peopl know first appear clas...
3                          good satch albumfour star
4  well hundr christma antholog one best ever mix...
(20000, 1)


### 1.4 WF, TFIDF, Ngrams

In [ ]:
#input file 
df = pd.read_csv('clean_train.csv', error_bad_lines = False)
df.head()
# check if null exists
df['text'].isnull().values.any()
for col in df.columns:
    n_nan = df[col].isnull().sum()
    if n_nan >0  :
        print(n_nan, col)
        
#dropna
# df_copy = df.dropna(axis = 0)

#replace NaN with unknown 
#replace with unknown 
for col in df.columns:
    df[col] = df[col].fillna('Unknown')

for col in df.columns:
    n_nan = df[col].isnull().sum()
    if n_nan >0  :
        print(n_nan, col)

61 text


In [ ]:
import copy
df_copy = copy.deepcopy(df)
df_copy.shape

(150000, 2)

In [ ]:
df_copy = df_copy.rename(columns={'overall': 'true_rating'})
df_copy.head()

,text,true_rating
0,contemporari jazz soul perform make twodecad c...,5.0
1,good idea put pop orchestr music matrix reload...,5.0
2,great collect carol king songsa musthav carol ...,5.0
3,album brilliant piec jazz fusion start finish ...,5.0
4,wait long time dvd releas watch yearold vh di...,5.0


In [ ]:
#changes begin###############################
#### train+test 
df_test = pd.read_csv('clean_test.csv', low_memory = False)
df_test = df_test.rename(columns={'overall': 'true_rating'})
for col in df_test.columns:
    df_test[col] = df_test[col].fillna('Unknown')
y = df_copy[['true_rating']]
df_copy = df_copy.drop(['true_rating'], axis=1)
df_copy 

,text
0,contemporari jazz soul perform make twodecad c...
1,good idea put pop orchestr music matrix reload...
2,great collect carol king songsa musthav carol ...
3,album brilliant piec jazz fusion start finish ...
4,wait long time dvd releas watch yearold vh di...
...,...
149995,someth wesley voic make swoon littl bit voic h...
149996,mani record verdi requiem oftenask question se...
149997,best overal album revisionist fallout follow g...
149998,believ upon time artist like enchant could see...


In [ ]:
df_concat = pd.concat([df_copy, df_test])
df_concat

,text
0,contemporari jazz soul perform make twodecad c...
1,good idea put pop orchestr music matrix reload...
2,great collect carol king songsa musthav carol ...
3,album brilliant piec jazz fusion start finish ...
4,wait long time dvd releas watch yearold vh di...
...,...
19995,good christian scott simpli take breath away b...
19996,congratul clan xymox butcher classic song wri...
19997,im happi cher video dvd couldnt rememb caus ch...
19998,agre comment make maxwel johnson regard sati b...


#### 1.4.1 WF max_feature = 4000

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

class WF_CV(object):
    def __init__(self,dataFrame):
        super().__init__()
        self.dataFrame = dataFrame
    
    def convert(self, text_col):
        return self.wf_convert(text_col)

    def wf_convert(self, text_col):
        wf_vectorizer = CountVectorizer(stop_words = 'english', max_features = 4000, max_df = 0.8)
        X = wf_vectorizer.fit_transform(self.dataFrame[text_col])
        return X

In [ ]:

WF_cvt = WF_CV(df_concat)
X_concat_wf= WF_cvt.wf_convert('text')
X_concat_wf 

<170000x4000 sparse matrix of type '<class 'numpy.int64'>'
	with 6575228 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy.sparse import csr_matrix

X_concat_dense = csr_matrix.todense(X_concat_wf )
X_concat_dense.shape

(170000, 4000)

In [ ]:
X_wf_dense = X_concat_dense[0:150000, :]
X_test_test_wf_dense = X_concat_dense[150000: , :]

X_wf_dense.shape, X_test_test_wf_dense.shape

((150000, 4000), (20000, 4000))

In [ ]:
X_wf = csr_matrix(X_wf_dense)
X_test_test_wf = csr_matrix(X_test_test_wf_dense)
X_wf

<150000x4000 sparse matrix of type '<class 'numpy.longlong'>'
	with 5815371 stored elements in Compressed Sparse Row format>

In [ ]:
#WF data split
X_train_wf, X_test_wf , y_train_wf, y_test_wf = train_test_split( X_wf, y, test_size = 0.3, random_state= 0, shuffle = False )

#### 1.4.2 TFIDF

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer 
class TFIDF_Convert(object):
    def __init__(self,dataFrame):
        super().__init__()
        self.dataFrame = dataFrame
    
    def convert(self, text_col):
        return self.tfidf_convert(text_col)

    def tfidf_convert(self, text_col):
        tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 15000, max_df = 0.8)
        X = tfidf_vectorizer.fit_transform(self.dataFrame[text_col])
        
        return X

In [ ]:
TF_cvt = TFIDF_Convert(df_concat)
X_concat_tf= TF_cvt.tfidf_convert('text')
X_concat_tf

In [ ]:
from scipy.sparse import csr_matrix

X_concat_dense_tf = csr_matrix.todense(X_concat_tf )
X_concat_dense_tf.shape

In [ ]:
X_tf_dense = X_concat_dense_tf[0:150000, :]
X_test_test_tf_dense = X_concat_dense_tf[150000: , :]

X_tf_dense.shape, X_test_test_tf_dense.shape

In [ ]:
X_tf = csr_matrix(X_tf_dense)
X_test_test_tf = csr_matrix(X_test_test_tf_dense)
X_tf

In [ ]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split( X_tf, y, test_size = 0.3, random_state = 0, shuffle = False)

##### Train

##### Test

#### 1.4.3 Ngrams

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
class ngram_Convert(object):
    def __init__(self,dataFrame):
        super().__init__()
        self.dataFrame = dataFrame
    
    def convert(self, text_col):
        return self.ngram_convert(text_col)

    def ngram_convert(self, text_col):
        cv_ngram = CountVectorizer(max_features=4000,binary=True,ngram_range=(2, 2)) 
        X = cv_ngram.fit_transform(self.dataFrame[text_col])

        return X


In [ ]:
ngram_cvt = ngram_Convert(df_copy)
X_ngram = ngram_cvt.convert('text')
X_ngram 

In [ ]:
X_train_ngram, X_test_ngram, y_train_ngram, y_test_ngram = train_test_split( X_ngram, y, test_size = 0.3, random_state = 0, shuffle = False)

## 2. Model Implementation
We will use the following Models:


1. Decision Tree

2. Linear SVR

3. Ridge Regression

4. SGD Regressor

5. Xgboost Regressor

6. Random Forest

### 2.1 DT

#### wf

In [ ]:
parameters={'min_samples_split': [10, 20, 40], 
                    'max_depth': [2, 8, 16], 
                    'min_samples_leaf': [20, 40, 100], 
                    'max_leaf_nodes': [5, 20, 100]}
model =  DecisionTreeRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_wf, y_train_wf)

print('WF features')
print('The optimal parameters alpha for DT is: \n', clf.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error
model_opt_wf_dt =DecisionTreeRegressor(min_samples_split=10 , max_depth=16, min_samples_leaf=40,max_leaf_nodes=100)
model_opt_wf_dt.fit(X_train_wf, y_train_wf)
y_pred_wf = model_opt_wf_dt.predict(X_test_wf)

print('For WF feature')
print('MSE:',mean_squared_error(y_test_wf, y_pred_wf))

For WF feature
MSE: 0.8235100005409548


#### tfidf

In [ ]:
parameters={'min_samples_split': [10, 20, 40], 
                    'max_depth': [2, 8, 16], 
                    'min_samples_leaf': [20, 40, 100], 
                    'max_leaf_nodes': [5, 20, 100]}
model = DecisionTreeRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_tf, y_train_tf)

print('For TFIDF features')
print('The optimal parameters alpha for DT model is: \n', clf.best_params_)
# print('With cross-validation accuracy of: {}%'.format(round(clf.best_score_*100,2)))

In [ ]:
model_opt_tf_dt = DecisionTreeRegressor(min_samples_split= 10, max_depth=16, min_samples_leaf=20,max_leaf_nodes=100)
model_opt_tf_dt.fit(X_train_tf, y_train_tf)
y_pred_tf = model_opt_tf_dt.predict(X_test_tf)
# acc_tf = accuracy_score(y_test_tf, y_pred_tf)

print('For TFIDF feature')
# print('Test accuracy of multinomial naive bayes model on WF is: {}%'.format(round(acc_tf *100, 2)))
print('MSE:', mean_squared_error(y_test_tf, y_pred_tf))

#### Ngrams

In [ ]:
parameters={'min_samples_split': [10, 20, 40], 
                    'max_depth': [2, 8, 16], 
                    'min_samples_leaf': [20, 40, 100], 
                    'max_leaf_nodes': [5, 20, 100]}
model = DecisionTreeRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_ngram, y_train_ngram)

print('ngrams features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)

In [ ]:
model_opt_ngram_dt = DecisionTreeRegressor(min_samples_split= 40, max_depth=16, min_samples_leaf=20,max_leaf_nodes=100)
model_opt_ngram_dt.fit(X_train_ngram, y_train_ngram)
y_pred_ngram = model_opt_ngram_dt.predict(X_test_ngram)

print('For N-gram feature')
print('MSE:',mean_squared_error(y_test_ngram, y_pred_ngram))

### 2.2 LinearSVR

#### wf

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'C': [0.001, 0.01,.01, 1.0, 10, 100],
            'gammas' = [0.001, 0.01, 0.1, 1]}
model = LinearSVR()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_wf, y_train_wf)

print('WF features')
print('The optimal parameters alpha for Ridge model is: \n', clf.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error
model_opt_wf_svr =LinearSVR(C=0.01)
model_opt_wf_svr.fit(X_train_wf, y_train_wf)
y_pred_wf = model_opt_wf_svr.predict(X_test_wf)

print('For WF feature')
print('MSE:',mean_squared_error(y_test_wf, y_pred_wf))

For WF feature
MSE: 0.9011257181086527


#### tfidf

In [ ]:
model = LinearSVR()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_tf, y_train_tf)

print('For TFIDF features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)
# print('With cross-validation accuracy of: {}%'.format(round(clf.best_score_*100,2)))

In [ ]:
model_opt_tf_svr = LinearSVR(C = 1.0)
model_opt_tf_svr.fit(X_train_tf, y_train_tf)
y_pred_tf = model_opt_tf_svr.predict(X_test_tf)
# acc_tf = accuracy_score(y_test_tf, y_pred_tf)

print('For TFIDF feature')
# print('Test accuracy of multinomial naive bayes model on WF is: {}%'.format(round(acc_tf *100, 2)))
print('MSE:', mean_squared_error(y_test_tf, y_pred_tf))

#### n-grams

In [ ]:
model = LinearSVR()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_ngram, y_train_ngram)

print('WF features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)

In [ ]:
model_opt_ngram_svr = LinearSVR(C=1.0)
model_opt_ngram_svr.fit(X_train_ngram, y_train_ngram)
y_pred_ngram = model_opt_ngram_svr.predict(X_test_ngram)

print('For N-gram feature')
print('MSE:',mean_squared_error(y_test_ngram, y_pred_ngram))

### 2.3 Ridge Regression

#### wf


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

parameters = {'alpha': [0.001, 0.01,.01, 1.0, 10, 100]}
model = Ridge()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_wf, y_train_wf)

print('WF features')
print('The optimal parameters alpha for Ridge model is: \n', clf.best_params_)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
model_opt_wf_ridge =Ridge(alpha=100)
model_opt_wf_ridge.fit(X_train_wf, y_train_wf)
y_pred_wf = model_opt_wf_ridge.predict(X_test_wf)

print('For WF feature')
print('MSE:',mean_squared_error(y_test_wf, y_pred_wf))

For WF feature
MSE: 0.7348134557885461


In [ ]:
X_train_wf

#### tfidf

In [ ]:
parameters = {'alpha': [0.001, 0.01,.01, 1.0, 10, 100]}
model = Ridge()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_tf, y_train_tf)

print('For TFIDF features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)
# print('With cross-validation accuracy of: {}%'.format(round(clf.best_score_*100,2)))

In [ ]:
model_opt_tf_ridge = Ridge(alpha = 1.0)
model_opt_tf_ridge.fit(X_train_tf, y_train_tf)
y_pred_tf = model_opt_tf_ridge.predict(X_test_tf)
# acc_tf = accuracy_score(y_test_tf, y_pred_tf)

print('For TFIDF feature')
# print('Test accuracy of multinomial naive bayes model on WF is: {}%'.format(round(acc_tf *100, 2)))
print('MSE:', mean_squared_error(y_test_tf, y_pred_tf))

#### n-grams

In [ ]:
parameters = {'alpha': [0.001, 0.01,.01, 1.0, 10, 100]}
model = Ridge()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_ngram, y_train_ngram)

print('ngrams features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)

In [ ]:
model_opt_ngram_ridge = Ridge(alpha=100)
model_opt_ngram_ridge.fit(X_train_ngram, y_train_ngram)
y_pred_ngram = model_opt_ngram_ridge.predict(X_test_ngram)

print('For N-gram feature')
print('MSE:',mean_squared_error(y_test_ngram, y_pred_ngram))

### 2.4 SGD

#### wf

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
parameters = {'alpha': [0.001, 0.01,.01, 1.0, 10, 100]}
model = SGDRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_wf, y_train_wf)

print('WF features')
print('The optimal parameters alpha for Ridge model is: \n', clf.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
model_opt_wf_SGD =SGDRegressor(alpha=1.0)
model_opt_wf_SGD.fit(X_train_wf, y_train_wf)
y_pred_wf = model_opt_wf_SGD.predict(X_test_wf)

print('For WF feature')
print('MSE:',mean_squared_error(y_test_wf, y_pred_wf))

For WF feature
MSE: 0.9129196614198615


#### tfidf

In [ ]:
from sklearn.metrics import mean_squared_error
model_opt_tf_SGD =SGDRegressor(alpha=1.0)
model_opt_tf_SGD.fit(X_train_tf, y_train_tf)
y_pred_tf = model_opt_tf_SGD.predict(X_test_tf)

print('For TFIDF feature')
print('MSE:',mean_squared_error(y_test_tf, y_pred_tf))

In [ ]:
from sklearn.metrics import mean_squared_error
model_opt_tf_SGD = SGDRegressor(alpha = 0.001)
model_opt_tf_SGD.fit(X_train_tf, y_train_tf)
y_pred_tf = model_opt_tf_SGD.predict(X_test_tf)
# acc_tf = accuracy_score(y_test_tf, y_pred_tf)

print('For TFIDF feature')
print('MSE:', mean_squared_error(y_test_tf, y_pred_tf))

#### n-grams

In [ ]:
parameters = {'alpha': [0.001, 0.01,.01, 1.0, 10, 100]}
model = SGDRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_ngram, y_train_ngram)

print('ngrams features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)

In [ ]:
model_opt_ngram_SGD = SGDRegressor(alpha=0.001)
model_opt_ngram_SGD.fit(X_train_ngram, y_train_ngram)
y_pred_ngram = model_opt_ngram_SGD.predict(X_test_ngram)

print('For N-gram feature')
print('MSE:',mean_squared_error(y_test_ngram, y_pred_ngram))

### 2.5 XG

#### wf

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor


In [ ]:
parameters = {'max_depth': [2,4,6,8,10]}
model = XGBRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_wf, y_train_wf)

print('WF features')
print('The optimal parameters alpha for XGBoost model is: \n', clf.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

model_opt_wf_XG =XGBRegressor(max_depth=6)
model_opt_wf_XG.fit(X_train_wf, y_train_wf)
y_pred_wf = model_opt_wf_XG.predict(X_test_wf)

print('For WF feature')
print('Test MSE:',mean_squared_error(y_test_wf, y_pred_wf))

[23:58:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
For WF feature
Test MSE: 0.6949168924427146


#### tfidf

In [ ]:
parameters = {'max_depth': [2,4,6,8,10]}
model = XGBRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_tf, y_train_tf)

print('For TFIDF features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)
# print('With cross-validation accuracy of: {}%'.format(round(clf.best_score_*100,2)))

In [ ]:
model_opt_tf_XG = XGBRegressor(max_depth = 6, gamma =  )
model_opt_tf_XG.fit(X_train_tf, y_train_tf)
y_pred_tf = model_opt_tf_XG.predict(X_test_tf)


print('For TFIDF feature')
# print('Test accuracy of multinomial naive bayes model on WF is: {}%'.format(round(acc_tf *100, 2)))
print('MSE:', mean_squared_error(y_test_tf, y_pred_tf))

#### n-grams

In [ ]:
parameters = {'max_depth': [2,4,6,8,10]}
model = XGBRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_ngram, y_train_ngram)

print('ngrams features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)

In [ ]:
model_opt_ngram = XGBRegressor(max_depth = 8)
model_opt_ngram.fit(X_train_ngram, y_train_ngram)
y_pred_ngram = model_opt_ngram.predict(X_test_ngram)

print('For N-gram feature')
print('MSE:',mean_squared_error(y_test_ngram, y_pred_ngram))

### 2.6 RF

#### wf

In [ ]:
from sklearn.ensemble import RandomForestRegressor
parameters = {'max_depth':[3,5,7,9],
             'min_samples_split':[5,10,20]}
model = RandomForestRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_wf, y_train_wf)

print('WF features')
print('The optimal parameters alpha for Ridge model is: \n', clf.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model_opt_wf_RF =RandomForestRegressor()
model_opt_wf_RF.fit(X_train_wf, y_train_wf)
y_pred_wf = model_opt_wf_RF.predict(X_test_wf)

print('For WF feature')
print('MSE:',mean_squared_error(y_test_wf, y_pred_wf))

#### tfidf

In [ ]:
parameters = {'max_depth':[3,5,7,9],
             'min_samples_split':[5,10,20]}
model = RandomForestRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_tf, y_train_tf)

print('For TFIDF features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)
# print('With cross-validation accuracy of: {}%'.format(round(clf.best_score_*100,2)))

In [ ]:
model_opt_tf_RF = RandomForestRegressor( )
model_opt_tf_RF.fit(X_train_tf, y_train_tf)
y_pred_tf = model_opt_tf_RF.predict(X_test_tf)
# acc_tf = accuracy_score(y_test_tf, y_pred_tf)

print('For TFIDF feature')
# print('Test accuracy of multinomial naive bayes model on WF is: {}%'.format(round(acc_tf *100, 2)))
print('MSE:', mean_squared_error(y_test_tf, y_pred_tf))

#### n-grams

In [ ]:
parameters = {'max_depth':[3,5,7,9],
             'min_samples_split':[5,10,20]}
model = RandomForestRegressor()
clf = GridSearchCV(model, parameters, cv =10, n_jobs = -1)
clf.fit(X_train_ngram, y_train_ngram)

print('ngrams features')
print('The optimal parameters alpha for multinomial naive bayes model is: \n', clf.best_params_)

In [ ]:
model_opt_ngram_RF = RandomForestRegressor(max_depth)
model_opt_ngram_RF.fit(X_train_ngram, y_train_ngram)
y_pred_ngram = model_opt_ngram_RF.predict(X_test_ngram)

print('For N-gram feature')
print('MSE:',mean_squared_error(y_test_ngram, y_pred_ngram))

# 3. Test Prediction

### 3.1 Decision Tree



In [ ]:
# #wf
y_pred_test_wf_dt = model_opt_wf_dt.predict(X_test_test_wf)

#tfidf
y_pred_test_tf_dt = model_opt_tf_dt.predict(X_test_test_tf)

#ngram
y_pred_test_ngram_dt = model_opt_ngram_dt.predict(X_test_ngram)

In [ ]:
df_pairs_wf.to_csv('test_Keras_prediction_COMBINEDwf.csv', sep = ',', index = False)


### 3.2. Linear SVR



In [ ]:
# #wf
y_pred_test_wf_svr = model_opt_wf_svr.predict(X_test_test_wf)

#tfidf
y_pred_test_tf_svr = model_opt_tf_svr.predict(X_test_test_tf)

#ngram
y_pred_test_ngram_svr = model_opt_ngram_svr.predict(X_test_ngram)

###3.3 Ridge Regression



In [ ]:
# #wf
y_pred_test_wf_ridge = model_opt_wf_ridge.predict(X_test_test_wf)

#tfidf
y_pred_test_tf_ridge = model_opt_tf_ridge.predict(X_test_test_tf)

#ngram
y_pred_test_ngram_ridge = model_opt_ngram_ridge.predict(X_test_ngram)

### 3.4 SGD

In [ ]:
# #wf
y_pred_test_wf_SGD = model_opt_wf_SGD.predict(X_test_test_wf)

#tfidf
y_pred_test_tf_SGD = model_opt_tf_SGD.predict(X_test_test_tf)

#ngram
y_pred_test_ngram_SGD = model_opt_ngram_SGD.predict(X_test_ngram)

### 3.5 XG

In [ ]:
# #wf
y_pred_test_wf_XG = model_opt_wf_XG.predict(X_test_test_wf)

#tfidf
y_pred_test_tf_XG = model_opt_tf_XG.predict(X_test_test_tf)

#ngram
y_pred_test_ngram_XG = model_opt_ngram_XG.predict(X_test_ngram)

### 3.6 RF

In [ ]:
# #wf
y_pred_test_wf_RF = model_opt_wf_RF.predict(X_test_test_wf)

#tfidf
y_pred_test_tf_RF = model_opt_tf_RF.predict(X_test_test_tf)

#ngram
y_pred_test_ngram_RF = model_opt_ngram_RF.predict(X_test_ngram)